In [1]:
import pandas as pd
import seaborn as sns 

In [ ]:
file_path = '../data/England CSV.csv'
df = pd.read_csv(file_path)

In [3]:
#Get initial look at data 

df.head()

,Date,Season,HomeTeam,AwayTeam,FTH Goals,FTA Goals,FT Result,HTH Goals,HTA Goals,HT Result,...,H Fouls,A Fouls,H Corners,A Corners,H Yellow,A Yellow,H Red,A Red,Display_Order,League
0,16/01/2025,2024/25,Ipswich Town,Brighton & Hove Albion,0,2,A,0.0,1.0,A,...,13.0,14.0,1.0,9.0,2.0,2.0,0.0,0.0,20250116,Premier League
1,16/01/2025,2024/25,Man United,Southampton,3,1,H,0.0,1.0,A,...,7.0,10.0,4.0,4.0,1.0,3.0,0.0,0.0,20250116,Premier League
2,15/01/2025,2024/25,Everton,Aston Villa,0,1,A,0.0,0.0,D,...,17.0,10.0,8.0,5.0,2.0,1.0,0.0,0.0,20250115,Premier League
3,15/01/2025,2024/25,Leicester,Crystal Palace,0,2,A,0.0,0.0,D,...,7.0,6.0,4.0,3.0,0.0,0.0,0.0,0.0,20250115,Premier League
4,15/01/2025,2024/25,Newcastle,Wolves,3,0,H,1.0,0.0,H,...,10.0,13.0,4.0,2.0,0.0,2.0,0.0,0.0,20250115,Premier League


In [4]:
#Get a look at the columns to see potential features 

df.columns

Index(['Date', 'Season', 'HomeTeam', 'AwayTeam', 'FTH Goals', 'FTA Goals',
       'FT Result', 'HTH Goals', 'HTA Goals', 'HT Result', 'Referee',
       'H Shots', 'A Shots', 'H SOT', 'A SOT', 'H Fouls', 'A Fouls',
       'H Corners', 'A Corners', 'H Yellow', 'A Yellow', 'H Red', 'A Red',
       'Display_Order', 'League'],
      dtype='object')

- Predicting exact scores would be a challenge so most likely use 'FT Result' as label
- Shots, corners, and cards are standout features already 
- No columns for team form or standing which would hinder prediction performance 
- Can possibly create new column for team form using grouping 
- Also create a column for total points

## Adding in total points 

In [33]:
#Work on calculating points won from home each gameweek 

def get_points(result, type = 'home'):
    if type == 'home':
        if result == 'H':
            return 3
        elif result == 'D':
            return 1
        else:
            return 0
    elif type == 'away':
        if result == 'A':
            return 3
        elif result == 'D':
            return 1
        else:
            return 0

home_df = df[['Date', 'Season', 'HomeTeam', 'FT Result']].rename(
    columns={'HomeTeam': 'Team', 'FT Result': 'Result'}
)

home_df['Points'] = home_df['Result'].apply(get_points)
home_df['Date'] = pd.to_datetime(home_df['Date'], format='%d/%m/%Y')
home_df['Venue'] = 'Home'

home_df.head()

,Date,Season,Team,Result,Points,Venue
0,2025-01-16,2024/25,Ipswich Town,A,0,Home
1,2025-01-16,2024/25,Man United,H,3,Home
2,2025-01-15,2024/25,Everton,A,0,Home
3,2025-01-15,2024/25,Leicester,A,0,Home
4,2025-01-15,2024/25,Newcastle,H,3,Home


In [34]:
#Do same for points won away from home 

away_df = df[['Date', 'Season', 'AwayTeam', 'FT Result']].rename(
    columns={'AwayTeam': 'Team', 'FT Result': 'Result'}
)

away_df['Points'] = away_df['Result'].apply(get_points, type='away')
away_df['Date'] = pd.to_datetime(away_df['Date'], format='%d/%m/%Y')
away_df['Venue'] = 'Away'

away_df.head()

,Date,Season,Team,Result,Points,Venue
0,2025-01-16,2024/25,Brighton & Hove Albion,A,3,Away
1,2025-01-16,2024/25,Southampton,H,0,Away
2,2025-01-15,2024/25,Aston Villa,A,3,Away
3,2025-01-15,2024/25,Crystal Palace,A,3,Away
4,2025-01-15,2024/25,Wolves,H,0,Away


In [35]:
away_and_home_df = pd.concat([home_df, away_df]).sort_values(by='Date').reset_index(drop=True)
away_and_home_df.head(10)

,Date,Season,Team,Result,Points,Venue
0,1993-08-14,1993/94,Oldham,A,0,Home
1,1993-08-14,1993/94,Newcastle,A,0,Home
2,1993-08-14,1993/94,Chelsea,A,0,Home
3,1993-08-14,1993/94,Arsenal,A,0,Home
4,1993-08-14,1993/94,Southampton,A,0,Home
5,1993-08-14,1993/94,Sheffield United,H,3,Home
6,1993-08-14,1993/94,Liverpool,H,3,Home
7,1993-08-14,1993/94,Aston Villa,H,3,Home
8,1993-08-14,1993/94,Man City,D,1,Home
9,1993-08-14,1993/94,QPR,H,0,Away


In [36]:
#Calculate the cumulative points 

away_and_home_df['Cumulative Points'] = away_and_home_df.groupby(['Season', 'Team'])['Points'].cumsum()
away_and_home_df.head(50)

,Date,Season,Team,Result,Points,Venue,Cumulative Points
0,1993-08-14,1993/94,Oldham,A,0,Home,0
1,1993-08-14,1993/94,Newcastle,A,0,Home,0
2,1993-08-14,1993/94,Chelsea,A,0,Home,0
3,1993-08-14,1993/94,Arsenal,A,0,Home,0
4,1993-08-14,1993/94,Southampton,A,0,Home,0
5,1993-08-14,1993/94,Sheffield United,H,3,Home,3
6,1993-08-14,1993/94,Liverpool,H,3,Home,3
7,1993-08-14,1993/94,Aston Villa,H,3,Home,3
8,1993-08-14,1993/94,Man City,D,1,Home,1
9,1993-08-14,1993/94,QPR,H,0,Away,0


In [38]:
#Calculate venue specific cumulative points
away_and_home_df['Venue Cumulative Points'] = away_and_home_df.groupby(['Season', 'Team', 'Venue'])['Points'].cumsum()
away_and_home_df.head(50)

,Date,Season,Team,Result,Points,Venue,Cumulative Points,Venue Cumulative Points
0,1993-08-14,1993/94,Oldham,A,0,Home,0,0
1,1993-08-14,1993/94,Newcastle,A,0,Home,0,0
2,1993-08-14,1993/94,Chelsea,A,0,Home,0,0
3,1993-08-14,1993/94,Arsenal,A,0,Home,0,0
4,1993-08-14,1993/94,Southampton,A,0,Home,0,0
5,1993-08-14,1993/94,Sheffield United,H,3,Home,3,3
6,1993-08-14,1993/94,Liverpool,H,3,Home,3,3
7,1993-08-14,1993/94,Aston Villa,H,3,Home,3,3
8,1993-08-14,1993/94,Man City,D,1,Home,1,1
9,1993-08-14,1993/94,QPR,H,0,Away,0,0


In [68]:
#create a copy of the original dataframe to add new features to

copy_df = df.copy()
copy_df['Date'] = pd.to_datetime(copy_df['Date'], format='%d/%m/%Y')
copy_df.head()

,Date,Season,HomeTeam,AwayTeam,FTH Goals,FTA Goals,FT Result,HTH Goals,HTA Goals,HT Result,...,H Fouls,A Fouls,H Corners,A Corners,H Yellow,A Yellow,H Red,A Red,Display_Order,League
0,2025-01-16,2024/25,Ipswich Town,Brighton & Hove Albion,0,2,A,0.0,1.0,A,...,13.0,14.0,1.0,9.0,2.0,2.0,0.0,0.0,20250116,Premier League
1,2025-01-16,2024/25,Man United,Southampton,3,1,H,0.0,1.0,A,...,7.0,10.0,4.0,4.0,1.0,3.0,0.0,0.0,20250116,Premier League
2,2025-01-15,2024/25,Everton,Aston Villa,0,1,A,0.0,0.0,D,...,17.0,10.0,8.0,5.0,2.0,1.0,0.0,0.0,20250115,Premier League
3,2025-01-15,2024/25,Leicester,Crystal Palace,0,2,A,0.0,0.0,D,...,7.0,6.0,4.0,3.0,0.0,0.0,0.0,0.0,20250115,Premier League
4,2025-01-15,2024/25,Newcastle,Wolves,3,0,H,1.0,0.0,H,...,10.0,13.0,4.0,2.0,0.0,2.0,0.0,0.0,20250115,Premier League


In [63]:
home_stats = away_and_home_df[away_and_home_df['Venue'] == 'Home']

away_stats = away_and_home_df[away_and_home_df['Venue'] == 'Away']
home_stats.head()

,Date,Season,Team,Result,Points,Venue,Cumulative Points,Venue Cumulative Points
0,1993-08-14,1993/94,Oldham,A,0,Home,0,0
1,1993-08-14,1993/94,Newcastle,A,0,Home,0,0
2,1993-08-14,1993/94,Chelsea,A,0,Home,0,0
3,1993-08-14,1993/94,Arsenal,A,0,Home,0,0
4,1993-08-14,1993/94,Southampton,A,0,Home,0,0


In [69]:
copy_df = pd.merge(copy_df, home_stats[['Date', 'Team', 'Venue Cumulative Points']],
                                left_on=['Date', 'HomeTeam'],
                                right_on=['Date', 'Team'],
                                how='left')
copy_df.head()

,Date,Season,HomeTeam,AwayTeam,FTH Goals,FTA Goals,FT Result,HTH Goals,HTA Goals,HT Result,...,H Corners,A Corners,H Yellow,A Yellow,H Red,A Red,Display_Order,League,Team,Venue Cumulative Points
0,2025-01-16,2024/25,Ipswich Town,Brighton & Hove Albion,0,2,A,0.0,1.0,A,...,1.0,9.0,2.0,2.0,0.0,0.0,20250116,Premier League,Ipswich Town,0
1,2025-01-16,2024/25,Man United,Southampton,3,1,H,0.0,1.0,A,...,4.0,4.0,1.0,3.0,0.0,0.0,20250116,Premier League,Man United,16
2,2025-01-15,2024/25,Everton,Aston Villa,0,1,A,0.0,0.0,D,...,8.0,5.0,2.0,1.0,0.0,0.0,20250115,Premier League,Everton,10
3,2025-01-15,2024/25,Leicester,Crystal Palace,0,2,A,0.0,0.0,D,...,4.0,3.0,0.0,0.0,0.0,0.0,20250115,Premier League,Leicester,9
4,2025-01-15,2024/25,Newcastle,Wolves,3,0,H,1.0,0.0,H,...,4.0,2.0,0.0,2.0,0.0,0.0,20250115,Premier League,Newcastle,20


In [70]:
copy_df.rename(columns={'Venue Cumulative Points': 'Home Cumulative Points'}, inplace=True)
copy_df.drop(columns=['Team'], inplace=True)
copy_df.head()

,Date,Season,HomeTeam,AwayTeam,FTH Goals,FTA Goals,FT Result,HTH Goals,HTA Goals,HT Result,...,A Fouls,H Corners,A Corners,H Yellow,A Yellow,H Red,A Red,Display_Order,League,Home Cumulative Points
0,2025-01-16,2024/25,Ipswich Town,Brighton & Hove Albion,0,2,A,0.0,1.0,A,...,14.0,1.0,9.0,2.0,2.0,0.0,0.0,20250116,Premier League,0
1,2025-01-16,2024/25,Man United,Southampton,3,1,H,0.0,1.0,A,...,10.0,4.0,4.0,1.0,3.0,0.0,0.0,20250116,Premier League,16
2,2025-01-15,2024/25,Everton,Aston Villa,0,1,A,0.0,0.0,D,...,10.0,8.0,5.0,2.0,1.0,0.0,0.0,20250115,Premier League,10
3,2025-01-15,2024/25,Leicester,Crystal Palace,0,2,A,0.0,0.0,D,...,6.0,4.0,3.0,0.0,0.0,0.0,0.0,20250115,Premier League,9
4,2025-01-15,2024/25,Newcastle,Wolves,3,0,H,1.0,0.0,H,...,13.0,4.0,2.0,0.0,2.0,0.0,0.0,20250115,Premier League,20


In [71]:
copy_df = pd.merge(copy_df, away_stats[['Date', 'Team', 'Venue Cumulative Points']],
                                left_on=['Date', 'AwayTeam'],
                                right_on=['Date', 'Team'],
                                how='left')
copy_df.head()  

,Date,Season,HomeTeam,AwayTeam,FTH Goals,FTA Goals,FT Result,HTH Goals,HTA Goals,HT Result,...,A Corners,H Yellow,A Yellow,H Red,A Red,Display_Order,League,Home Cumulative Points,Team,Venue Cumulative Points
0,2025-01-16,2024/25,Ipswich Town,Brighton & Hove Albion,0,2,A,0.0,1.0,A,...,9.0,2.0,2.0,0.0,0.0,20250116,Premier League,0,Brighton & Hove Albion,3
1,2025-01-16,2024/25,Man United,Southampton,3,1,H,0.0,1.0,A,...,4.0,1.0,3.0,0.0,0.0,20250116,Premier League,16,Southampton,2
2,2025-01-15,2024/25,Everton,Aston Villa,0,1,A,0.0,0.0,D,...,5.0,2.0,1.0,0.0,0.0,20250115,Premier League,10,Aston Villa,13
3,2025-01-15,2024/25,Leicester,Crystal Palace,0,2,A,0.0,0.0,D,...,3.0,0.0,0.0,0.0,0.0,20250115,Premier League,9,Crystal Palace,13
4,2025-01-15,2024/25,Newcastle,Wolves,3,0,H,1.0,0.0,H,...,2.0,0.0,2.0,0.0,0.0,20250115,Premier League,20,Wolves,9


In [72]:
copy_df.rename(columns={'Venue Cumulative Points': 'Away Cumulative Points'}, inplace=True)
copy_df.drop(columns=['Team'], inplace=True)
copy_df.head()

,Date,Season,HomeTeam,AwayTeam,FTH Goals,FTA Goals,FT Result,HTH Goals,HTA Goals,HT Result,...,H Corners,A Corners,H Yellow,A Yellow,H Red,A Red,Display_Order,League,Home Cumulative Points,Away Cumulative Points
0,2025-01-16,2024/25,Ipswich Town,Brighton & Hove Albion,0,2,A,0.0,1.0,A,...,1.0,9.0,2.0,2.0,0.0,0.0,20250116,Premier League,0,3
1,2025-01-16,2024/25,Man United,Southampton,3,1,H,0.0,1.0,A,...,4.0,4.0,1.0,3.0,0.0,0.0,20250116,Premier League,16,2
2,2025-01-15,2024/25,Everton,Aston Villa,0,1,A,0.0,0.0,D,...,8.0,5.0,2.0,1.0,0.0,0.0,20250115,Premier League,10,13
3,2025-01-15,2024/25,Leicester,Crystal Palace,0,2,A,0.0,0.0,D,...,4.0,3.0,0.0,0.0,0.0,0.0,20250115,Premier League,9,13
4,2025-01-15,2024/25,Newcastle,Wolves,3,0,H,1.0,0.0,H,...,4.0,2.0,0.0,2.0,0.0,0.0,20250115,Premier League,20,9


In [ ]:
# Add overall cumulative points for home team
copy_df = pd.merge(copy_df, away_and_home_df[['Date', 'Team', 'Cumulative Points']],
                                left_on=['Date', 'HomeTeam'],
                                right_on=['Date', 'Team'],
                                how='left')
copy_df.head()

,Date,Season,HomeTeam,AwayTeam,FTH Goals,FTA Goals,FT Result,HTH Goals,HTA Goals,HT Result,...,H Yellow,A Yellow,H Red,A Red,Display_Order,League,Home Cumulative Points,Away Cumulative Points,Team,Cumulative Points
0,2025-01-16,2024/25,Ipswich Town,Brighton & Hove Albion,0,2,A,0.0,1.0,A,...,2.0,2.0,0.0,0.0,20250116,Premier League,0,3,Ipswich Town,0
1,2025-01-16,2024/25,Man United,Southampton,3,1,H,0.0,1.0,A,...,1.0,3.0,0.0,0.0,20250116,Premier League,16,2,Man United,26
2,2025-01-15,2024/25,Everton,Aston Villa,0,1,A,0.0,0.0,D,...,2.0,1.0,0.0,0.0,20250115,Premier League,10,13,Everton,17
3,2025-01-15,2024/25,Leicester,Crystal Palace,0,2,A,0.0,0.0,D,...,0.0,0.0,0.0,0.0,20250115,Premier League,9,13,Leicester,14
4,2025-01-15,2024/25,Newcastle,Wolves,3,0,H,1.0,0.0,H,...,0.0,2.0,0.0,0.0,20250115,Premier League,20,9,Newcastle,38


In [74]:
copy_df.rename(columns={'Cumulative Points': 'Home Team Overall Cumulative Points'}, inplace=True)
copy_df.drop(columns=['Team'], inplace=True)
copy_df.head()

,Date,Season,HomeTeam,AwayTeam,FTH Goals,FTA Goals,FT Result,HTH Goals,HTA Goals,HT Result,...,A Corners,H Yellow,A Yellow,H Red,A Red,Display_Order,League,Home Cumulative Points,Away Cumulative Points,Home Team Overall Cumulative Points
0,2025-01-16,2024/25,Ipswich Town,Brighton & Hove Albion,0,2,A,0.0,1.0,A,...,9.0,2.0,2.0,0.0,0.0,20250116,Premier League,0,3,0
1,2025-01-16,2024/25,Man United,Southampton,3,1,H,0.0,1.0,A,...,4.0,1.0,3.0,0.0,0.0,20250116,Premier League,16,2,26
2,2025-01-15,2024/25,Everton,Aston Villa,0,1,A,0.0,0.0,D,...,5.0,2.0,1.0,0.0,0.0,20250115,Premier League,10,13,17
3,2025-01-15,2024/25,Leicester,Crystal Palace,0,2,A,0.0,0.0,D,...,3.0,0.0,0.0,0.0,0.0,20250115,Premier League,9,13,14
4,2025-01-15,2024/25,Newcastle,Wolves,3,0,H,1.0,0.0,H,...,2.0,0.0,2.0,0.0,0.0,20250115,Premier League,20,9,38


In [75]:
copy_df = pd.merge(copy_df, away_and_home_df[['Date', 'Team', 'Cumulative Points']],
                                left_on=['Date', 'AwayTeam'],
                                right_on=['Date', 'Team'],
                                how='left')
copy_df.rename(columns={'Cumulative Points': 'Away Team Overall Cumulative Points'}, inplace=True)
copy_df.drop(columns=['Team'], inplace=True)
copy_df.head()

,Date,Season,HomeTeam,AwayTeam,FTH Goals,FTA Goals,FT Result,HTH Goals,HTA Goals,HT Result,...,H Yellow,A Yellow,H Red,A Red,Display_Order,League,Home Cumulative Points,Away Cumulative Points,Home Team Overall Cumulative Points,Away Team Overall Cumulative Points
0,2025-01-16,2024/25,Ipswich Town,Brighton & Hove Albion,0,2,A,0.0,1.0,A,...,2.0,2.0,0.0,0.0,20250116,Premier League,0,3,0,4
1,2025-01-16,2024/25,Man United,Southampton,3,1,H,0.0,1.0,A,...,1.0,3.0,0.0,0.0,20250116,Premier League,16,2,26,6
2,2025-01-15,2024/25,Everton,Aston Villa,0,1,A,0.0,0.0,D,...,2.0,1.0,0.0,0.0,20250115,Premier League,10,13,17,35
3,2025-01-15,2024/25,Leicester,Crystal Palace,0,2,A,0.0,0.0,D,...,0.0,0.0,0.0,0.0,20250115,Premier League,9,13,14,24
4,2025-01-15,2024/25,Newcastle,Wolves,3,0,H,1.0,0.0,H,...,0.0,2.0,0.0,0.0,20250115,Premier League,20,9,38,16


In [78]:
copy_df = copy_df.sort_values(by='Date').reset_index(drop=True)
copy_df.head()


,Date,Season,HomeTeam,AwayTeam,FTH Goals,FTA Goals,FT Result,HTH Goals,HTA Goals,HT Result,...,H Yellow,A Yellow,H Red,A Red,Display_Order,League,Home Cumulative Points,Away Cumulative Points,Home Team Overall Cumulative Points,Away Team Overall Cumulative Points
0,1993-08-14,1993/94,Oldham,Ipswich,0,3,A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,19930814,Premier League,0,3,0,3
1,1993-08-14,1993/94,West Ham,Wimbledon,0,2,A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,19930814,Premier League,0,3,0,3
2,1993-08-14,1993/94,Man City,Leeds,1,1,D,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,19930814,Premier League,1,1,1,1
3,1993-08-14,1993/94,Newcastle,Tottenham,0,1,A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,19930814,Premier League,0,3,0,3
4,1993-08-14,1993/94,Chelsea,Blackburn,1,2,A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,19930814,Premier League,0,3,0,3


In [79]:
#Initialize featuresdf that will contain all features for modeling
features_df = copy_df[['Date', 'Season', 'HomeTeam', 'AwayTeam',
                       'Home Cumulative Points', 'Away Cumulative Points',
                       'Home Team Overall Cumulative Points', 'Away Team Overall Cumulative Points']]
features_df.head(10)

,Date,Season,HomeTeam,AwayTeam,Home Cumulative Points,Away Cumulative Points,Home Team Overall Cumulative Points,Away Team Overall Cumulative Points
0,1993-08-14,1993/94,Oldham,Ipswich,0,3,0,3
1,1993-08-14,1993/94,West Ham,Wimbledon,0,3,0,3
2,1993-08-14,1993/94,Man City,Leeds,1,1,1,1
3,1993-08-14,1993/94,Newcastle,Tottenham,0,3,0,3
4,1993-08-14,1993/94,Chelsea,Blackburn,0,3,0,3
5,1993-08-14,1993/94,Aston Villa,QPR,3,0,3,0
6,1993-08-14,1993/94,Southampton,Everton,0,3,0,3
7,1993-08-14,1993/94,Liverpool,Sheffield Weds,3,0,3,0
8,1993-08-14,1993/94,Arsenal,Coventry,0,3,0,3
9,1993-08-14,1993/94,Sheffield United,Swindon,3,0,3,0
